In [1]:
from sklearn import datasets
iris = datasets.load_iris()
X_iris, y_iris = iris.data, iris.target

In [2]:
print (X_iris.shape, y_iris.shape)
print (X_iris[0], y_iris[0])

(150, 4) (150,)
[ 5.1  3.5  1.4  0.2] 0


In [7]:
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing

In [8]:
# Get dataset with only the first two attributes
X, y = X_iris[:, :2], y_iris

In [9]:
# Split the dataset into a training and a testing set
# Test set will be the 25% taken randomly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)
print (X_train.shape, y_train.shape)

(112, 2) (112,)


In [10]:
# Standardize the features
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)